In [0]:
spark.sql("""
CREATE OR REPLACE TABLE workspace.churn.churn_reason_raw AS
SELECT
  Partner AS customer_id,
  Churn,
  CustomerFeedback_clean,
  ai_extract(
    CONCAT(
      'Você é um analista de churn da operadora. ',
      'Leia o feedback e extraia: ',
      '(1) reason_short = motivo principal em 3 a 6 palavras; ',
      '(2) reason_long = explicação em uma frase. ',
      'Feedback: ',
      CustomerFeedback_clean
    ),
    array(
      'reason_short',
      'reason_long'
    )
  ) AS reason_struct
FROM workspace.churn.history_genie
WHERE Churn = 1
""")

In [0]:
%sql
SELECT
  customer_id,
  Churn,
  CustomerFeedback_clean,
  reason_struct.reason_short,
  reason_struct.reason_long
FROM workspace.churn.churn_reason_raw
LIMIT 20;

In [0]:
%sql
-- Top motivos curtos
SELECT
  reason_struct,
  COUNT(*) AS qtd
FROM workspace.churn.churn_reason_raw
GROUP BY reason_struct
ORDER BY qtd DESC
LIMIT 50;

Taxonomia com base no Short reason e Long Reason

1. **Better deal / pricing**  
   _Exemplos:_ encontrou oferta melhor, melhor preço, promoção concorrente, custo alto etc.

2. **Technical issues / speed**  
   _Exemplos:_ internet lenta, velocidades inconsistentes, problemas técnicos genéricos.

3. **Service reliability / outages**  
   _Exemplos:_ quedas frequentes, instabilidade, serviço “vai e volta”.

4. **Product / plan mismatch**  
   _Exemplos:_ plano sem internet, recurso esperado não incluído, tipo de serviço não atende uso.

5. **Payment / billing issues**  
   _Exemplos:_ método de pagamento inconveniente, cobrança, faturamento.

6. **Personal reasons**  
   _Exemplos:_ mudança, motivos pessoais genéricos.

7. **Other / unclear**  
   _Exemplos:_ o que não encaixar claramente acima.

In [0]:
spark.sql("""
CREATE OR REPLACE TABLE workspace.churn.churn_reason_final AS
SELECT
  customer_id,
  Churn,
  CustomerFeedback_clean,
  reason_struct.reason_short AS reason_short,
  reason_struct.reason_long  AS reason_long,
  ai_classify(
    CONCAT(
      'Motivo curto: ', reason_struct.reason_short, '. ',
      'Motivo detalhado: ', reason_struct.reason_long, '. ',
      'Classifique o motivo de churn em UMA das categorias abaixo.'
    ),
    array(
      'Better deal / pricing',
      'Technical issues / speed',
      'Service reliability / outages',
      'Product / plan mismatch',
      'Payment / billing issues',
      'Personal reasons',
      'Other / unclear'
    )
  ) AS churn_category
FROM workspace.churn.churn_reason_raw
""")

In [0]:
%sql
SELECT
  churn_category,
  COUNT(*) AS qtd
FROM workspace.churn.churn_reason_final
GROUP BY churn_category
ORDER BY qtd DESC;